In [1]:
import os
from dotenv import load_dotenv,find_dotenv
_=load_dotenv(find_dotenv())
google_api_key=os.environ['GEMINI_API_KEY']

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",api_key=google_api_key)

#testing
response=llm.invoke("Write capital of Bihar")
print(response.content)

c:\Users\panka\OneDrive\Desktop\Euron\GenAi\9.Level_1_Application_Development\Level-1-Apllication-Development\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Patna


we will create an app to extract structured info from unstructured text.

eg. we will have to extract name and last name and country of users that submit commnets on websites os our companay

## Defining what information we want to extract
* **We'll use Pydantic to define an schema to extract personal information**.
* Pydantic is a Python library used for data validation. It helps ensure that the data your program receives matches the format you expect, and it provides helpful error messages when the data doesn't conform to your specifications. Essentially, Pydantic allows you to enforce that the data structures in Python adhere to specific types and constraints, making your code more robust and error-resistant.
* **Document the attributes and the schema itself**: This information is sent to the LLM and is used to improve the quality of information extraction.
* Do not force the LLM to make up information! **We import Optional for the attributes allowing the LLM to output None if it doesn't know the answer**.
* When you use Optional in type hints, you are indicating that a variable can either be of a specified type or it can be None.

In [2]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(
        default=None, description="The name of the person"
    )
    lastname: Optional[str] = Field(
        default=None, description="The lastname of the person if known"
    )
    country: Optional[str] = Field(
        default=None, description="The country of the person if known"
    )

c:\Users\panka\OneDrive\Desktop\Euron\GenAi\9.Level_1_Application_Development\Level-1-Apllication-Development\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


Definig Extractor

In [29]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.pydantic_v1 import BaseModel, Field

# Defining a custom prompt to provide instructions and any additional context.
# 1) we can add examples into the prompt template to improve extraction quality
# 2) we can introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information as Name and country name  from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value."
        
        ),
        ("human", "{text}"),
    ]
)

In [30]:
chain=prompt | llm.with_structured_output(schema=Person)

In [33]:
comment = "I absolutely love this product! It's been a game-changer for my daily routine. The quality is top-notch and the customer service is outstanding. I've recommended it to all my friends and family. - Sarah Johnson, India"

In [34]:
chain.invoke({"text": comment})

Person(name='Sarah Johnson', lastname=None, country=None)

## Extraction of a list of entities rather than a single entity
* In real projects we will probably work with a large text that includes more than one user review. **We can extract the key data of several users by nesting Pydantic models**.
* See how the Data model definition is including the Person model. This is technically called "nesting" models.

In [35]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(
        default=None, description="The name of the person"
    )
    lastname: Optional[str] = Field(
        default=None, description="The lastname of the person if known"
    )
    country: Optional[str] = Field(
        default=None, description="The country of the person if known"
    )

class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [36]:
chain = prompt | llm.with_structured_output(schema=Data)

In [37]:
comment = "I'm so impressed with this product! It has truly transformed how I approach my daily tasks. The quality exceeds my expectations, and the customer support is truly exceptional. I've already suggested it to all my colleagues and relatives. - Emily Clarke, Canada"

In [38]:
chain.invoke({"text": comment})

Data(people=[Person(name='Emily Clarke', lastname=None, country=None)])

In [39]:
# Example input text that mentions multiple people
text_input = """
Alice Johnson from Canada recently reviewed a book she loved. Meanwhile, Bob Smith from the USA shared his insights on the same book in a different review. Both reviews were very insightful.
"""

response = chain.invoke({"text": text_input})

# Output the extracted data
response

Data(people=[Person(name='Alice Johnson', lastname=None, country=None)])